Question 2

Scenario: The "Strategic Tile Shatter" Game

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import time


# ALGORITHM SECTION - Dynamic Programming


class TileShatterSolver:
 
    def __init__(self, tile_multipliers):
        self.multipliers = tile_multipliers
        self.n = len(tile_multipliers)
        self.dp = [[0] * self.n for _ in range(self.n)]
        self.parent = [[-1] * self.n for _ in range(self.n)]
        
    def solve(self):
 
        if self.n == 0:
            return {'max_points': 0, 'sequence': [], 'steps': []}
        if self.n == 1:
            return {'max_points': 0, 'sequence': [0], 'steps': []}
        
        # Build DP table bottom-up by increasing interval length
        for length in range(3, self.n + 1):
            for i in range(self.n - length + 1):
                j = i + length - 1
                
                # Try shattering each tile k between i and j
                for k in range(i + 1, j):
                    # Get left and right multipliers (with out-of-bounds handling)
                    left_mult = self.multipliers[i - 1] if i - 1 >= 0 else 1
                    right_mult = self.multipliers[j + 1] if j + 1 < self.n else 1
                    
                    # Points for shattering tile k
                    points = left_mult * self.multipliers[k] * right_mult
                    
                    # Total points = points from left interval + right interval + current
                    total_points = self.dp[i][k] + self.dp[k][j] + points
                    
                    # Update if this is better
                    if total_points > self.dp[i][j]:
                        self.dp[i][j] = total_points
                        self.parent[i][j] = k
        
        # Reconstruct the optimal sequence
        sequence = []
        steps = []
        self._reconstruct_sequence(0, self.n - 1, list(range(self.n)), sequence, steps)
        
        return {
            'max_points': self.dp[0][self.n - 1],
            'sequence': sequence,
            'steps': steps
        }
    
    def _reconstruct_sequence(self, i, j, remaining, sequence, steps):
      
        if i >= j - 1:
            return
        
        k = self.parent[i][j]
        if k == -1:
            return
        
        # Recurse on left and right intervals first
        self._reconstruct_sequence(i, k, remaining, sequence, steps)
        self._reconstruct_sequence(k, j, remaining, sequence, steps)
        
        # Then shatter tile k
        if k not in sequence:
            # Calculate points for this shatter
            tiles_left = [idx for idx in remaining if idx not in sequence]
            points = self._calculate_shatter_points(k, tiles_left)
            
            steps.append({
                'tile_index': k,
                'tile_value': self.multipliers[k],
                'points': points,
                'remaining_before': tiles_left.copy()
            })
            sequence.append(k)
    
    def _calculate_shatter_points(self, shatter_idx, remaining):
        """Calculate points for shattering a specific tile."""
        # Find left neighbor
        left_mult = 1
        for idx in reversed(remaining):
            if idx < shatter_idx:
                left_mult = self.multipliers[idx]
                break
        
        # Find right neighbor
        right_mult = 1
        for idx in remaining:
            if idx > shatter_idx:
                right_mult = self.multipliers[idx]
                break
        
        return left_mult * self.multipliers[shatter_idx] * right_mult



# GUI SECTION 


class TileShatterGUI:
    
    def __init__(self, root):
        self.root = root
        self.root.title("Strategic Tile Shatter Game")
        self.root.geometry("900x700")
        self.root.configure(bg="#1a1a2e")
        
        # Game state
        self.tiles = []
        self.solution = None
        self.current_step = 0
        
        self._create_widgets()
        
    def _create_widgets(self):
        """Create all GUI widgets."""
        # Title
        title_frame = tk.Frame(self.root, bg="#1a1a2e")
        title_frame.pack(pady=20)
        
        title_label = tk.Label(
            title_frame,
            text="Strategic Tile Shatter",
            font=("Arial", 28, "bold"),
            bg="#1a1a2e",
            fg="#00d4ff"
        )
        title_label.pack()
        
        subtitle_label = tk.Label(
            title_frame,
            text="Dynamic Programming Solution",
            font=("Arial", 12),
            bg="#1a1a2e",
            fg="#7f8c8d"
        )
        subtitle_label.pack()
        
        # Input Frame
        input_frame = tk.Frame(self.root, bg="#16213e", relief=tk.RAISED, bd=2)
        input_frame.pack(pady=20, padx=40, fill=tk.X)
        
        tk.Label(
            input_frame,
            text="Enter Tile Multipliers (comma-separated):",
            font=("Arial", 12, "bold"),
            bg="#16213e",
            fg="white"
        ).pack(pady=10)
        
        self.input_entry = tk.Entry(
            input_frame,
            font=("Arial", 14),
            justify="center",
            bg="#0f3460",
            fg="white",
            insertbackground="white"
        )
        self.input_entry.pack(pady=5, padx=20, fill=tk.X)
        self.input_entry.insert(0, "3, 1, 5, 8")
        
        # Buttons Frame
        button_frame = tk.Frame(input_frame, bg="#16213e")
        button_frame.pack(pady=10)
        
        tk.Button(
            button_frame,
            text="Solve",
            font=("Arial", 12, "bold"),
            bg="#00d4ff",
            fg="white",
            command=self._solve,
            width=12,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            button_frame,
            text="Reset",
            font=("Arial", 12, "bold"),
            bg="#e74c3c",
            fg="white",
            command=self._reset,
            width=12,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=5)
        
        # Results Frame
        self.results_frame = tk.Frame(self.root, bg="#1a1a2e")
        self.results_frame.pack(pady=20, padx=40, fill=tk.BOTH, expand=True)
        
        # Maximum Points Display
        self.points_label = tk.Label(
            self.results_frame,
            text="Maximum Points: -",
            font=("Arial", 24, "bold"),
            bg="#1a1a2e",
            fg="#f39c12"
        )
        self.points_label.pack(pady=10)
        
        # Tiles Canvas
        self.canvas_frame = tk.Frame(self.results_frame, bg="#16213e", relief=tk.RAISED, bd=2)
        self.canvas_frame.pack(pady=10, fill=tk.BOTH, expand=True)
        
        tk.Label(
            self.canvas_frame,
            text="Tile Board:",
            font=("Arial", 14, "bold"),
            bg="#16213e",
            fg="white"
        ).pack(pady=5)
        
        self.tiles_canvas = tk.Canvas(
            self.canvas_frame,
            bg="#0f3460",
            height=100,
            highlightthickness=0
        )
        self.tiles_canvas.pack(pady=10, padx=20, fill=tk.X)
        
        # Steps Display
        steps_label = tk.Label(
            self.results_frame,
            text="Solution Steps:",
            font=("Arial", 14, "bold"),
            bg="#1a1a2e",
            fg="white"
        )
        steps_label.pack(pady=5)
        
        # Scrollable text widget for steps
        text_frame = tk.Frame(self.results_frame)
        text_frame.pack(pady=5, padx=20, fill=tk.BOTH, expand=True)
        
        scrollbar = tk.Scrollbar(text_frame)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.steps_text = tk.Text(
            text_frame,
            font=("Courier", 10),
            bg="#16213e",
            fg="#ecf0f1",
            height=10,
            yscrollcommand=scrollbar.set,
            wrap=tk.WORD
        )
        self.steps_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.config(command=self.steps_text.yview)
        
    def _solve(self):
        try:
            # Parse input
            input_text = self.input_entry.get()
            self.tiles = [int(x.strip()) for x in input_text.split(',')]
            
            if len(self.tiles) == 0:
                messagebox.showerror("Error", "Please enter at least one tile multiplier")
                return
            
            # Solve using DP
            solver = TileShatterSolver(self.tiles)
            self.solution = solver.solve()
            
            # Display results
            self._display_results()
            
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numbers separated by commas")
    
    def _display_results(self):
        """Display the solution results."""
        # Update points label
        self.points_label.config(
            text=f"Maximum Points: {self.solution['max_points']}"
        )
        
        # Draw tiles
        self._draw_tiles()
        
        # Display steps
        self._display_steps()
    
    def _draw_tiles(self):
        """Draw tiles on canvas."""
        self.tiles_canvas.delete("all")
        
        if not self.tiles:
            return
        
        canvas_width = self.tiles_canvas.winfo_width()
        if canvas_width <= 1:
            canvas_width = 800
        
        tile_width = 60
        tile_height = 60
        spacing = 20
        total_width = len(self.tiles) * tile_width + (len(self.tiles) - 1) * spacing
        start_x = (canvas_width - total_width) // 2
        
        for i, mult in enumerate(self.tiles):
            x = start_x + i * (tile_width + spacing)
            y = 20
            
            # Determine tile color based on state
            if self.solution and i in self.solution['sequence'][:self.current_step]:
                color = "#e74c3c"  # Shattered - red
                text_color = "white"
            else:
                color = "#00d4ff"  # Active - blue
                text_color = "white"
            
            # Draw tile
            self.tiles_canvas.create_rectangle(
                x, y, x + tile_width, y + tile_height,
                fill=color,
                outline="white",
                width=2
            )
            
            # Draw multiplier text
            self.tiles_canvas.create_text(
                x + tile_width // 2,
                y + tile_height // 2,
                text=str(mult),
                font=("Arial", 20, "bold"),
                fill=text_color
            )
            
            # Draw index
            self.tiles_canvas.create_text(
                x + tile_width // 2,
                y + tile_height + 10,
                text=f"i={i}",
                font=("Arial", 9),
                fill="#7f8c8d"
            )
    
    def _display_steps(self):
        """Display solution steps."""
        self.steps_text.delete(1.0, tk.END)
        
        if not self.solution or not self.solution['steps']:
            self.steps_text.insert(tk.END, "No solution available.")
            return
        
        total_points = 0
        
        for step_num, step in enumerate(self.solution['steps'], 1):
            tile_idx = step['tile_index']
            tile_val = step['tile_value']
            points = step['points']
            remaining = step['remaining_before']
            
            total_points += points
            
            # Format remaining tiles
            remaining_str = ', '.join([f"{self.tiles[i]}" for i in remaining])
            
            self.steps_text.insert(
                tk.END,
                f"Step {step_num}: Shatter tile {tile_idx} (value={tile_val})\n"
            )
            self.steps_text.insert(
                tk.END,
                f"  Remaining: [{remaining_str}]\n"
            )
            self.steps_text.insert(
                tk.END,
                f"  Points earned: {points}\n"
            )
            self.steps_text.insert(
                tk.END,
                f"  Total so far: {total_points}\n"
            )
            self.steps_text.insert(tk.END, "-" * 50 + "\n\n")
        
        self.steps_text.insert(
            tk.END,
            f"\n{'='*50}\n"
            f"FINAL TOTAL POINTS: {self.solution['max_points']}\n"
            f"{'='*50}\n"
        )
    
    def _reset(self):
        """Reset the game."""
        self.tiles = []
        self.solution = None
        self.current_step = 0
        self.points_label.config(text="Maximum Points: -")
        self.tiles_canvas.delete("all")
        self.steps_text.delete(1.0, tk.END)
        self.input_entry.delete(0, tk.END)
        self.input_entry.insert(0, "3, 1, 5, 8")


if __name__ == "__main__":
    root = tk.Tk()
    app = TileShatterGUI(root)
    root.mainloop()